---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd


doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)
df = pd.Series(doc)

#df.head(10)

In [ ]:
def date_sorter():
    import pandas as pd
    def year_EX(line):
        import re
        import pandas as pd 
        res=re.match(r'[\w\W]*[ ]*(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})[\w\W]*',line)
        if(res):
            #print(res.group(0))
            Day=res.group(1)
            if(len(Day)==1):
                Day="0"+Day
            Month=res.group(2)
            if(len(Month)==1):
                Month="0"+Month
            Year=res.group(3)
            if(len(Year)==2):
                Year="19"+Year
            return "{}-{}-{}".format(int(Year),Month,Day)
        res=re.match(r'[\w\W]*[ ]*\d{2}[\W ]*(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ ]*\d{2,4}', line)
        if(res):
            mesi=dict(zip("Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec".split('|'),[x for x in range(1,13)]))
            if(len(res.group(0).split(' '))> 2):
                (day, mese, anno)=res.group(0).split(' ')[-3:]
                day=day.replace('.','')
                if(len(day)>2):
                    temp=day
                    day=mese
                    mese=temp
                mese=str(mesi[mese[:3]])
                if(len(mese)==1):
                    mese="0"+mese
                if(len(day)==1):
                    day="0"+day  
            else:
               (mese, anno)=res.group(0).split(' ')[-2:]
               day='01'
               mese=str(mesi[mese[:3]])
               if(len(mese)==1):
                    mese="0"+mese

            return "{}-{}-{}".format(anno,mese,day)
        res=re.match(r'[\w\W]*(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.]? \d{2}[,.]? \d{4}', line)
        if(res):
            mesi=dict(zip("Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec".split('|'),[x for x in range(1,13)]))
            if(len(res.group(0).split(' '))> 2):
                (day, mese, anno)=res.group(0).split(' ')[-3:]
                day=day.replace('.','')
                if(len(day)>2):
                    temp=day
                    day=mese
                    mese=temp
                mese=str(mesi[mese[:3]])
                if(len(mese)==1):
                    mese="0"+mese
                if(len(day)==1):
                    day="0"+day  
            else:
               (mese, anno)=res.group(0).split(' ')[-2:]
               day='01'
               mese=str(mesi[mese[:3]])
               if(len(mese)==1):
                    mese="0"+mese

            return "{}-{}-{}".format(anno,mese,day)
        res=re.match(r'[\w\W]*(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,.]?[ ]+\d{4}', line)
        if(res):
            mesi=dict(zip("Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec".split('|'),[x for x in range(1,13)]))
            (mese, anno)=res.group(0).split(' ')[-2:]
            day='01'
            mese=mese.replace('(','').replace(')','').replace('pOct','Oct').replace('AFeb','Feb').replace('yAug','Aug').replace('lNovember','November').replace('"','').replace('IAug','Aug')
            mese=mese.replace('sSep','Sep').replace('2June','Jun').replace('sNovember','Nov').replace('.','')
            anno=anno.replace('(','').replace(')','')
            mese=str(mesi[mese[:3]])
            if(len(mese)==1):
                mese="0"+mese

            return "{}-{}-{}".format(anno,mese,day)  
        res=re.match(r'[\w\W]*[^0-9]?(\d{1,2})[/-](\d{4})[\w\W]*',line)
        if(res):

            Month=res.group(1)
            if(len(Month)==1):
                Month="0"+Month
            Year=res.group(2)
            Day='01'
            return "{}-{}-{}".format(int(Year),Month,Day)

        res=re.match(r'[\w\W]*(\d{4})[\w\W]*',line)
        if(res):
            res.group(1)
            return "{}-{}-{}".format(int(res.group(1)),'01','01')

        return "9999999"
    #finish function...
#    doc = []
#    with open('dates.txt') as file:
#        for line in file:
#            doc.append(line)
#    df = pd.Series(doc)
    df_ = pd.DataFrame(data=df, columns=['text'])
    df_['year']=df_['text'].apply(lambda x:year_EX(x))
    #df_['Ranks']=df_['year'].rank(ascending=False)
    #df1=date_sorter()    
    df_.reset_index(level=0, inplace=True)
    df2=df_[['year','index']].sort_values(by='year')
    df2['oo']=df2['index']
    a=[i for i in range(500)]
    df2.index=a
    df2=df2.sort_values('oo')
    answer= pd.Series(df2.index)
    return answer



In [ ]:
date_sorter()

In [8]:
date_sorter()

0      277
1      174
2        0
3       56
4      312
5       98
6       94
7      227
8      185
9        1
10     173
11      53
12     327
13      14
14     241
15     464
16     134
17      67
18     337
19      79
20     335
21     104
22     233
23      66
24     162
25     136
26     226
27      61
28       3
29     190
      ... 
470    141
471    338
472    447
473     45
474      5
475    487
476    213
477    281
478    268
479    309
480    476
481     26
482    229
483    292
484    386
485    193
486     16
487    253
488     70
489    163
490    415
491    437
492    177
493     82
494    371
495     95
496    407
497    430
498    401
499    106
Length: 500, dtype: int64